In [1]:
from datetime import datetime
from sklearn.manifold import TSNE
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torch.nn.init as init
from torch.autograd import Variable
import argparse
import copy
import numpy as np
import time
import os
from PIL import Image
from utils.dataloader_all import *
from utils.auc_test import *
from utils import new_transforms

In [2]:
#why this is taking 
root_dir = "/beegfs/sb3923/DeepCancer/alldata/AlldataTilesSorted/"
num_classes = 9
tile_dict_path = '/beegfs/sb3923/DeepCancer/alldata/Alldata_FileMappingDict.p'
imgSize = 299

transform = transforms.Compose([new_transforms.Resize((imgSize,imgSize)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [176]:
#if want to rerun things
test_data = TissueData2(root_dir, 'test', transform = transform, metadata=False)
classes = test_data.classes
print(classes)

os.chdir('/scratch/jmw784/capstone/deep-cancer')
import pickle
pickle.dump( test_data, open( "test_data.p", "wb" ) )
pickle.dump( classes, open( "classes.p", "wb" ) )

Loading from: ST_Normal_Kidney


KeyboardInterrupt: 

In [3]:
os.chdir('/scratch/jmw784/capstone/deep-cancer')
import pickle
test_data = pickle.load( open( "test_data.p", "rb" ))
classes = pickle.load( open( "classes.p", "rb" ))


In [4]:
c = pickle.load( open( "classes.p", "rb" ))

In [5]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

In [6]:
with open(tile_dict_path, 'rb') as f:
    tile_dict = pickle.load(f)

In [7]:
def get_tile_probability(tile_path):
    # Some tiles are empty with no path, return nan
    if tile_path == '':
        return np.full(num_classes, np.nan)
    
    tile_path = root_dir + tile_path
 
    with open(tile_path, 'rb') as f:
        with Image.open(f) as img:
            img = img.convert('RGB')
            
    img = transform(img).unsqueeze(0)
    
    # Turn output into probabilities with softmax
    var_img = Variable(img, volatile=True).cuda()
    output = F.softmax(model(var_img)[0]).data.squeeze(0)
    output = output.cpu()
    
    #viz = (model(var_img)[1]).squeeze(0)#torch.FloatTensor of size 1x5184
    #viz = viz.cpu()
    #print(output.numpy())
    return output.numpy()#, viz.data.numpy()
    

In [8]:
def get_tile_probability2(tile_path):
    # Some tiles are empty with no path, return nan
    if tile_path == '':
        return np.full(5184, np.nan)
    
    tile_path = root_dir + tile_path
    #print(tile_path)
 
    with open(tile_path, 'rb') as f:
        with Image.open(f) as img:
            img = img.convert('RGB')
            
    img = transform(img).unsqueeze(0)
    
    # Turn output into probabilities with softmax
    var_img = Variable(img, volatile=True).cuda()
    viz = (model(var_img)[1]).squeeze(0)#torch.FloatTensor of size 1x5184
    viz = viz.cpu()
    #print(viz.data.numpy())
    return viz.data.numpy() #numpy.ndarray

In [167]:
file_list = test_data.filenames

for file in file_list:
        tile_paths, label = tile_dict[file]
        #print(tile_paths)
        
        
folder = classes[label]

def add_folder(tile_path):
    if tile_path == '':
        return ''
    else:
        #print(folder + '/' + tile_path)
        #print(tile_path)
        #print(folder)
        return folder + '/' + tile_path
        
    
    
add_folder_v = np.vectorize(add_folder)
tile_paths = add_folder_v(tile_paths)


m=(tile_paths[6])
print(m)


prob_v2 = np.vectorize(get_tile_probability2, otypes=[np.ndarray])
lastlayer = prob_v2(m)
       
print(lastlayer)

['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' ''
 '' '' ''
 'TCGA-KIRP/test_TCGA-2Z-A9JK-01A-01-TS1.13B13EB1-C0D1-41FF-86C5-80422490E330_78_6.jpeg'
 '' '' ''
 'TCGA-KIRP/test_TCGA-2Z-A9JK-01A-01-TS1.13B13EB1-C0D1-41FF-86C5-80422490E330_82_6.jpeg'
 'TCGA-KIRP/test_TCGA-2Z-A9JK-01A-01-TS1.13B13EB1-C0D1-41FF-86C5-80422490E330_83_6.jpeg'
 'TCGA-KIRP/test_TCGA-2Z-A9JK-01A-01-TS1.13B13EB1-C0D1-41FF-86C5-80422490E330_84_6.jpeg'
 'TCGA-KIRP/test_TCGA-2Z-A9JK-01A-01-TS1.13B13EB1-C0D1-41FF-86C5-80422490E330_85_6.jpeg'
 'TCGA-KIRP/test_TCGA-2Z-A9JK-01A-01-TS1.13B13EB1-C0D1-41FF-86C5-80422490E330_86_6.jpeg'
 'TCGA-KIRP/test_TCGA-2Z-A9JK-01A-01-TS1.13B13EB1-C0D1-41FF-86C5-80422490E330_87_6.jpeg'
 'TCGA-KIRP/test_TCGA-2Z-A9JK-01A-01-TS1.13B13EB1-C0D1-41FF-86C5-80422490E330_88_6.jpeg'
 'TCGA-KIRP/test_TCGA-2Z-A9JK-01A-01-TS1

In [21]:
def aggregate(file_list, method):

    """
    Given a list of files, return scores for each class according to the
    method and labels for those files.
    @param file_list: A list of file paths to do predictions on
    @param method: 'average' - returns the average probability score across
                               all tiles for that file
                   'max' - predicts each tile to be the class of the maximum
                           score, and returns the proportion of tiles for
                           each class
    @return: a ndarray of class probabilities for all files in the list
             a ndarray of the labels
    """

    model.eval()
    predictions = []
    true_labels = []
    last_layer = []

    for file in file_list:
        tile_paths, label = tile_dict[file]

        folder = classes[label]

        def add_folder(tile_path):
            if tile_path == '':
                return ''
            else:
                return folder + '/' + tile_path

        # Add the folder for the class name in front
        add_folder_v = np.vectorize(add_folder)
        tile_paths0 = add_folder_v(tile_paths)
        

        # Get the probability array for the file
        prob_v = np.vectorize(get_tile_probability, otypes=[np.ndarray])
        probabilities = prob_v(tile_paths0)
        
        
        tile_paths1 = add_folder_v(tile_paths)
        
        
        #print(probabilities)
        #lastlayer = prob_v2(tile_paths)
       
        #print(probabilities[0])
        #print(type(probabilities))
        #print(probabilities.shape)
        #m=(tile_paths[6])
        #print(m)
        
        prob_v2 = np.vectorize(get_tile_probability2, otypes=[np.ndarray])
        lastlayer = prob_v2(tile_paths1)
        
        #print(lastlayer[0])
        #print(type(lastlayer))
        #print(lastlayer.shape)


        #probabilities
        probabilities = np.stack(probabilities.flat)
        prediction = np.nanmean(probabilities, axis = 0)
        
        tile_label = np.argmax(probabilities,axis=1)
        
        
        #last layer
        last_layer_weights = np.stack(lastlayer.flat)
        
        a = last_layer_weights[np.ix_(label == tile_label),:]
        #print((np.squeeze(a)).ndim)
        
        #print(last_layer_weights.shape)
        #print(np.array(label == tile_label))
        #print(last_layer_weights.shape[np.array(label == tile_label),:])
        if (np.squeeze(a)).ndim>1:
            lastlayerweights = np.nanmean(np.squeeze(a), axis = 0)
            print(lastlayerweights.shape)
        
            predictions.append(prediction)
            true_labels.append(label)
            last_layer.append(lastlayerweights)
        else:
            print('no tile is correctly classified')
        
    return np.array(predictions), np.array(true_labels), np.array(last_layer)

In [22]:
class BasicConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, pool, **kwargs):
        super(BasicConv2d, self).__init__()

        self.pool = pool
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)
        self.relu = nn.LeakyReLU()
        
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        x = self.conv(x)

        if self.pool:
            x = F.max_pool2d(x, 2)
        
        x = self.relu(x)
        x = self.bn(x)
        x = self.dropout(x)
        return x

# Define model
class cancer_CNN(nn.Module):
    def __init__(self, nc, imgSize, ngpu):
        super(cancer_CNN, self).__init__()
        self.nc = nc
        self.imgSize = imgSize
        self.ngpu = ngpu
        self.data = 'all'
        self.conv1 = BasicConv2d(nc, 16, False, kernel_size=5, padding=1, stride=2, bias=True)
        self.conv2 = BasicConv2d(16, 32, False, kernel_size=3, bias=True)
        self.conv3 = BasicConv2d(32, 64, True, kernel_size=3, padding=1, bias=True)
        self.conv4 = BasicConv2d(64, 64, True, kernel_size=3, padding=1, bias=True)
        self.conv5 = BasicConv2d(64, 128, True, kernel_size=3, padding=1, bias=True)
        self.conv6 = BasicConv2d(128, 64, True, kernel_size=3, padding=1, bias=True)
        self.linear = nn.Linear(5184, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = x.view(x.size(0), -1)
        llw = x
        x = self.linear(x)
        return x, llw

In [ ]:
model = cancer_CNN(3, imgSize, 1)
model.cuda()
model_path = "/scratch/jmw784/capstone/deep-cancer/experiments/train_all_downsample_2/epoch_17.pth"
state_dict = torch.load(model_path, map_location=lambda storage, loc: storage)
model.load_state_dict(state_dict)
predictions, labels, lastlayer = aggregate(test_data.filenames, method='average')


(5184,)
(5184,)
(5184,)
(5184,)
(5184,)
(5184,)
(5184,)
(5184,)
(5184,)
(5184,)
(5184,)
(5184,)
(5184,)
(5184,)
